In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
test = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])

dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])


In [2]:
diff = dat.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
dat['solve_time'] = diff['Timestamp']

In [3]:
# 시험 난이도(?)로 추청되는 특성 따로 분류
dat['b_category'] = dat['assessmentItemID'].str[2]
# 시험지로 추청되는 특성 따로 분류. 뒤에 것이 좋을 것 같은데 일단 제출을 한 것으로 써놓음.
dat['test_category'] = dat['assessmentItemID'].str[2] + dat['assessmentItemID'].str[4:7] # dat['assessmentItemID'].str[4:7]
dat['problem_id'] = dat['assessmentItemID'].str[-3:]

In [4]:
diff['Timestamp'] = dat['solve_time']
diff['Timestamp'] = pd.qcut(diff['Timestamp'],5)
diff['Timestamp'] = diff['Timestamp'].astype("str")
dat['category_st_qcut_5'] = diff['Timestamp']

In [5]:
dat['solve_time'] = dat['solve_time'].astype(int)

In [6]:
dat['last_answerCode'] = dat.groupby("userID")['answerCode'].shift(1).fillna(1)
dat['last_answerCode2'] = dat.groupby("userID")['answerCode'].shift(2).fillna(1)
dat['last_answerCode3'] = dat.groupby("userID")['answerCode'].shift(3).fillna(1)
dat['last_answerCode4'] = dat.groupby("userID")['answerCode'].shift(4).fillna(1)
dat['last_answerCode5'] = dat.groupby("userID")['answerCode'].shift(5).fillna(1)
dat['last_answerCode6'] = dat.groupby("userID")['answerCode'].shift(6).fillna(1)
dat['last_answerCode7'] = dat.groupby("userID")['answerCode'].shift(7).fillna(1)
dat['last_answerCode8'] = dat.groupby("userID")['answerCode'].shift(8).fillna(1)
dat['last_answerCode9'] = dat.groupby("userID")['answerCode'].shift(9).fillna(1)
dat['last_answerCode10'] = dat.groupby("userID")['answerCode'].shift(10).fillna(1)

In [7]:
dat['last_answerCode'] = dat['last_answerCode'].astype(int)
dat['last_answerCode2'] = dat['last_answerCode2'].astype(int)
dat['last_answerCode3'] = dat['last_answerCode3'].astype(int)
dat['last_answerCode4'] = dat['last_answerCode4'].astype(int)
dat['last_answerCode5'] = dat['last_answerCode5'].astype(int)
dat['last_answerCode6'] = dat['last_answerCode6'].astype(int)
dat['last_answerCode7'] = dat['last_answerCode7'].astype(int)
dat['last_answerCode8'] = dat['last_answerCode8'].astype(int)
dat['last_answerCode9'] = dat['last_answerCode9'].astype(int)
dat['last_answerCode10'] = dat['last_answerCode10'].astype(int)

In [8]:
def elo(df):
    def get_new_theta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + learning_rate_theta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - learning_rate_beta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * sigmoid(theta - beta)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...", flush=True)

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            ),
            total=len(answers_df),
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        return student_parameters, item_parameters

    df["left_asymptote"] = 1/4

    print(f"Dataset of shape {df.shape}")
    print(f"Columns are {list(df.columns)}")

    student_parameters, item_parameters = estimate_parameters(df)

    
    t = [
        student_parameters[student]["theta"]
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]
    b = [
        item_parameters[item]["beta"]
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]
    #theta represents the global level of the student
    df['theta'] = t

    #beta represents the difficulty of the item 
    df['beta'] = b

    return df


In [9]:
# ELO Function 적용
dat = elo(dat)

# 필요없는 column 제거
dat = dat.drop(columns=["left_asymptote"])


Dataset of shape (2526700, 22)
Columns are ['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'solve_time', 'b_category', 'test_category', 'problem_id', 'category_st_qcut_5', 'last_answerCode', 'last_answerCode2', 'last_answerCode3', 'last_answerCode4', 'last_answerCode5', 'last_answerCode6', 'last_answerCode7', 'last_answerCode8', 'last_answerCode9', 'last_answerCode10', 'left_asymptote']
Parameter estimation is starting...


100%|██████████| 2526700/2526700 [00:17<00:00, 148552.52it/s]


Theta & beta estimations on assessmentItemID are completed.


In [11]:
dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,...,last_answerCode3,last_answerCode4,last_answerCode5,last_answerCode6,last_answerCode7,last_answerCode8,last_answerCode9,last_answerCode10,theta,beta
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,6,6001,001,...,1,1,1,1,1,1,1,1,0.075428,-3.449985
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6,6001,002,...,1,1,1,1,1,1,1,1,0.075428,-3.102043
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8,6,6001,003,...,1,1,1,1,1,1,1,1,0.075428,-2.284439
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7,6,6001,004,...,1,1,1,1,1,1,1,1,0.075428,-3.173193
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7,6,6001,005,...,1,1,1,1,1,1,1,1,0.075428,-2.665952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24,3,3071,005,...,0,0,1,1,1,1,1,1,-0.107730,1.644793
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,4,4165,001,...,1,0,0,1,1,1,1,1,-0.107730,-0.149751
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11,4,4165,002,...,0,1,0,0,1,1,1,1,-0.107730,-0.090214
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46,4,4165,003,...,0,0,1,0,0,1,1,1,-0.107730,-0.887353


In [12]:
dat.reset_index(inplace=True)

In [13]:
dat['theta'].dtype

dtype('float64')

In [14]:
dat['beta'].dtype

dtype('float64')

In [16]:
dat['year'] = dat['Timestamp'].dt.year # 연도 정보
dat['month'] =  dat['Timestamp'].dt.month # 월 정보
dat['day'] =  dat['Timestamp'].dt.day # 일 정보
dat['hour'] =  dat['Timestamp'].dt.hour # 시간 정보

dat.to_csv('/opt/ml/input/data/FE/FE_total_2_elo.csv', index=False)